In [11]:
# given a list of DCs, and a wrong repair(hand-picked), return the responsibility of the DCs

### 1. run holoclean and return wrongly repaired cells 

In [33]:
from examples.holoclean_repair_example import main
import psycopg2
import pandas as pd
import copy
pd.set_option('display.max_rows', None)

In [38]:
# connection to db 
conn = psycopg2.connect(dbname="holo", user="holocleanuser", password="abcd1234")
cols = ["ProviderNumber","HospitalName","Address1","Address2","Address3","City","State","ZipCode","CountyName","PhoneNumber","HospitalType","HospitalOwner",
        "EmergencyService","Condition","MeasureCode","MeasureName","Score","Sample","Stateavg"]
cur = conn.cursor()
conn.autocommit=True

In [41]:
import pprint

In [51]:
cur.execute("SELECT COLUMN_NAME from information_schema.columns WHERE table_schema = 'public' AND table_name = 'adult'");
cur.fetchone()

('_tid_',)

In [35]:

# drop preexisted repaired records 
select_old_repairs_q = """
SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES 
WHERE TABLE_NAME LIKE 'hospital_repaired_%' AND TABLE_TYPE = 'BASE TABLE'
"""
cur.execute(select_old_repairs_q)


In [36]:
for records in cur.fetchall():
    drop_q = f"drop table if exists {records[0]}"
    cur.execute(drop_q)

In [37]:
main('dc_finder_hospital_rules_5', initial_training=True)

16:29:57 - [DEBUG] - initiating session with parameters: {'db_user': 'holocleanuser', 'db_pwd': 'abcd1234', 'db_host': 'localhost', 'db_name': 'holo', 'threads': 1, 'timeout': 180000, 'seed': 45, 'learning_rate': 0.001, 'optimizer': 'adam', 'epochs': 10, 'weight_decay': 0.01, 'momentum': 0.0, 'batch_size': 1, 'weak_label_thresh': 0.99, 'domain_thresh_1': 0, 'domain_thresh_2': 0, 'max_domain': 10000, 'cor_strength': 0.6, 'nb_cor_strength': 0.8, 'feature_norm': False, 'weight_norm': False, 'estimator_epochs': 3, 'estimator_batch_size': 32, 'verbose': True, 'bias': False, 'print_fw': True, 'debug_mode': False}
16:29:57 - [WARNING] - Dropping the following null column from the dataset: 'Address2'
16:29:57 - [WARNING] - Dropping the following null column from the dataset: 'Address3'
16:29:57 - [ INFO] - Loaded 1000 rows with 18000 cells
16:29:58 - [DEBUG] - Time to create index: 0.00 secs
16:29:58 - [DEBUG] - Time to create index: 0.00 secs
16:29:58 - [DEBUG] - Time to create index: 0.00 se

16:30:09 - [DEBUG] - Starting to execute query SELECT _vid_, val_id, count(*) violations FROM   "hospital" as t1, "hospital" as t2, pos_values as t3 WHERE  t1._tid_ != t2._tid_   AND  t1._tid_ = t3._tid_   AND  t3.attribute = 'Stateavg'   AND  TRUE   AND  t3.rv_val = t2."Stateavg" GROUP BY _vid_, val_id with id 3
16:30:09 - [DEBUG] - Time to execute query with id 3: 0.01 secs
16:30:09 - [DEBUG] - Time to execute 4 queries: 0.03 secs
16:30:10 - [DEBUG] - DONE featurization.
16:30:10 - [DEBUG] - generating weak labels...
16:30:10 - [DEBUG] - Time to execute query: 0.02 secs
100%|██████████| 1390/1390 [00:00<00:00, 60722.44it/s]
16:30:10 - [DEBUG] - DONE generating weak labels.
16:30:10 - [DEBUG] - generating mask...
16:30:10 - [DEBUG] - Time to execute query: 0.01 secs
100%|██████████| 4773/4773 [00:00<00:00, 87274.67it/s]
16:30:10 - [DEBUG] - DONE generating mask.
16:30:10 - [ INFO] - DONE setting up featurized dataset.
16:30:10 - [DEBUG] - Time to featurize data: 6.99 secs
16:30:10 - [

16:30:17 - [DEBUG] - Time to store featurizer weights: 0.00 secs
16:30:18 - [DEBUG] - Time to create index: 0.01 secs
16:30:18 - [DEBUG] - Time to create index: 0.00 secs
16:30:18 - [ INFO] - DONE Loading hospital_clean.csv
16:30:18 - [DEBUG] - Time to evaluate repairs: 0.76 secs
16:30:18 - [DEBUG] - Time to execute query: 0.00 secs
16:30:18 - [DEBUG] - Time to execute query: 0.00 secs
16:30:18 - [DEBUG] - Preparing to execute 17 queries.
16:30:18 - [DEBUG] - Starting to execute query SELECT count(*) FROM  "hospital" as t1, "hospital_clean" as t2 WHERE t1._tid_ = t2._tid_   AND t2._attribute_ = 'ProviderNumber'   AND t1."ProviderNumber" != t2._value_ with id 0
16:30:18 - [DEBUG] - Time to execute query with id 0: 0.00 secs
16:30:18 - [DEBUG] - Starting to execute query SELECT count(*) FROM  "hospital" as t1, "hospital_clean" as t2 WHERE t1._tid_ = t2._tid_   AND t2._attribute_ = 'HospitalName'   AND t1."HospitalName" != t2._value_ with id 1
16:30:18 - [DEBUG] - Time to execute query wi

16:30:18 - [DEBUG] - Time to execute query with id 5: 0.00 secs
16:30:18 - [DEBUG] - Starting to execute query SELECT COUNT(*) FROM   (SELECT t2._tid_, t2._attribute_, t2._value_      FROM "hospital" as t1, "hospital_clean" as t2     WHERE t1._tid_ = t2._tid_       AND t2._attribute_ = 'CountyName'       AND t1."CountyName" != t2._value_ ) as errors, inf_values_dom as repairs WHERE errors._tid_ = repairs._tid_   AND errors._attribute_ = repairs.attribute   AND errors._value_ = repairs.rv_value with id 6
16:30:18 - [DEBUG] - Time to execute query with id 6: 0.00 secs
16:30:18 - [DEBUG] - Starting to execute query SELECT COUNT(*) FROM   (SELECT t2._tid_, t2._attribute_, t2._value_      FROM "hospital" as t1, "hospital_clean" as t2     WHERE t1._tid_ = t2._tid_       AND t2._attribute_ = 'PhoneNumber'       AND t1."PhoneNumber" != t2._value_ ) as errors, inf_values_dom as repairs WHERE errors._tid_ = repairs._tid_   AND errors._attribute_ = repairs.attribute   AND errors._value_ = repairs

In [17]:
### 2. run holoclean and return wrongly repaired ddcells 

In [18]:
union_sql = """
SELECT 'before_clean' AS type, * from  hospital
union all 
SELECT 'after_clean' AS type, * from hospital_repaired
order by _tid_
"""
df_union_before_and_after = pd.read_sql(union_sql, conn) 

In [19]:
df_union_before_and_after

,type,_tid_,ProviderNumber,HospitalName,Address1,City,State,ZipCode,CountyName,PhoneNumber,HospitalType,HospitalOwner,EmergencyService,Condition,MeasureCode,MeasureName,Score,Sample,Stateavg
0,after_clean,0,10018,callahan eye foundation hospital,1720 university blvd,birmingham,al,35233,jefferson,2053258100,acute care hospitals,voluntary non-profit - private,yes,surgical infection prevention,scip-card-2,surgery patients who were taking heart drugs c...,_nan_,_nan_,al_scip-card-2
1,before_clean,0,10018,callahan eye foundation hospital,1720 university blvd,birmingham,al,35233,jefferson,2053258100,acute care hospitals,voluntary non-profit - private,yes,surgical infection prevention,scip-card-2,surgery patients who were taking heart drugs c...,_nan_,_nan_,al_scip-card-2
2,after_clean,1,10018,callahan eye foundation hospital,1720 university blvd,birmingham,al,35233,jefferson,2053258100,acute care hospitals,voluntary non-profit - private,yes,surgical infection prevention,scip-inf-1,surgery patients who were given an antibiotic ...,_nan_,_nan_,al_scip-inf-1
3,before_clean,1,10018,callahan eye foundation hospital,1720 university blvd,birmingham,al,35233,jefferson,2053258100,acute care hospitals,voluntary non-profit - private,yes,surgical infection prevention,scip-inf-1,surgery patients who were given an antibiotic ...,_nan_,_nan_,al_scip-inf-1
4,before_clean,2,10018,callahan eye foundation hospital,1720 university blvd,birmingham,al,35233,jefferson,2053258100,acute care hospitals,voluntary non-profit - private,yes,surgical infection prevention,scip-inf-2,surgery patients who were given the right kin...,_nan_,_nan_,al_scip-inf-2
5,after_clean,2,10018,callahan eye foundation hospital,1720 university blvd,birmingham,al,35233,jefferson,2053258100,acute care hospitals,voluntary non-profit - private,yes,surgical infection prevention,scip-inf-2,surgery patients who were given the right kin...,_nan_,_nan_,al_scip-inf-2
6,before_clean,3,10018,callahan eye foundation hospital,1720 university blvd,birminghxm,al,35233,jefferson,2053258100,acute care hospitals,voluntary non-profit - private,yes,surgical infection prevention,scip-inf-3,surgery patients whose preventive antibiotics ...,_nan_,_nan_,al_scip-inf-3
7,after_clean,3,10018,callahan eye foundation hospital,1720 university blvd,birminghxm,al,35233,jefferson,2053258100,acute care hospitals,voluntary non-profit - private,yes,surgical infection prevention,scip-inf-3,surgery patients whose preventive antibiotics ...,_nan_,_nan_,al_scip-inf-3
8,before_clean,4,10018,callahan eye foundation hospital,1720 university blvd,birmingham,al,35233,jefferson,2053258100,acute care hospitals,voluntary non-profit - private,yes,surgical infection prevention,scip-inf-4,all heart surgery patients whose blood sugar (...,_nan_,_nan_,al_scip-inf-4
9,after_clean,4,10018,callahan eye foundation hospital,1720 university blvd,birmingham,al,35233,jefferson,2053258100,acute care hospitals,voluntary non-profit - private,yes,surgical infection prevention,scip-inf-4,all heart surgery patients whose blood sugar (...,_nan_,_nan_,al_scip-inf-4


In [20]:
j = 0
for i in range(0,1000):
    if(j%100==0):
        print(j)
    j+=1
    df_row = pd.DataFrame(columns=['type']+cols)
    df_row.loc[0,'type'] = 'ground_truth'
    df_row.loc[0,'_tid_'] = i
    q = f"""
    SELECT * FROM hospital_clean WHERE _tid_={i} 
    """
    df_for_one_row = pd.read_sql(q,conn)
    for index, row in df_for_one_row.iterrows():
        df_row.loc[0, f"{row['_attribute_']}"] = row['_value_']
    df_union_before_and_after = df_union_before_and_after.append(df_row, ignore_index=True)

0
100
200
300
400
500
600
700
800
900


In [21]:
repaired_dict = {x:[] for x in cols}

In [22]:
# iterate this dataframe to find all wrong predictions and list them to choose
grouped = df_union_before_and_after.groupby('_tid_')
k = 0

for name, group in grouped:
    tid = pd.to_numeric(group.iloc[0]['_tid_'], downcast="integer")
    if(k%100==0):
        print(k)
    for c in cols:
        if(group[group['type']=='after_clean'][c].to_string(index=False)\
           != group[group['type']=='before_clean'][c].to_string(index=False)):
            repaired_dict[c].append(tid)
    k+=1

0
100
200
300
400
500
600
700
800
900


In [23]:
repaired_dict

{'ProviderNumber': [],
 'HospitalName': [],
 'Address1': [],
 'Address2': [],
 'Address3': [],
 'City': [],
 'State': [],
 'ZipCode': [],
 'CountyName': [],
 'PhoneNumber': [],
 'HospitalType': [],
 'HospitalOwner': [493.0],
 'EmergencyService': [],
 'Condition': [],
 'MeasureCode': [97.0,
  108.0,
  120.0,
  177.0,
  190.0,
  264.0,
  349.0,
  391.0,
  398.0,
  409.0,
  464.0,
  495.0,
  580.0,
  639.0,
  673.0,
  676.0,
  691.0,
  694.0,
  718.0,
  729.0,
  735.0,
  749.0,
  788.0,
  865.0],
 'MeasureName': [],
 'Score': [],
 'Sample': [],
 'Stateavg': [56.0,
  69.0,
  193.0,
  213.0,
  244.0,
  248.0,
  269.0,
  334.0,
  347.0,
  370.0,
  373.0,
  387.0,
  432.0,
  433.0,
  435.0,
  436.0,
  437.0,
  438.0,
  439.0,
  440.0,
  441.0,
  442.0,
  443.0,
  444.0,
  445.0,
  446.0,
  448.0,
  449.0,
  450.0,
  451.0,
  477.0,
  555.0,
  564.0,
  567.0,
  622.0,
  646.0,
  678.0,
  721.0,
  760.0,
  764.0,
  787.0,
  796.0,
  827.0,
  920.0]}

In [24]:
for d,v in repaired_dict.items():
    print(f"{d}: {len(repaired_dict[d])}")

ProviderNumber: 0
HospitalName: 0
Address1: 0
Address2: 0
Address3: 0
City: 0
State: 0
ZipCode: 0
CountyName: 0
PhoneNumber: 0
HospitalType: 0
HospitalOwner: 1
EmergencyService: 0
Condition: 0
MeasureCode: 24
MeasureName: 0
Score: 0
Sample: 0
Stateavg: 44


In [25]:
wrong_dict = {x:[] for x in cols}

In [26]:
# iterate this dataframe to find all wrong predictions and list them to choose
grouped = df_union_before_and_after.groupby('_tid_')
k = 0

for name, group in grouped:
    tid = pd.to_numeric(group.iloc[0]['_tid_'], downcast="integer")
    if(k%100==0):
        print(k)
    for c in cols:
        if((group[group['type']=='after_clean'][c].to_string(index=False)\
           != group[group['type']=='ground_truth'][c].to_string(index=False)\
           ) and (group[group['type']=='after_clean'][c].to_string(index=False)\
           != group[group['type']=='before_clean'][c].to_string(index=False))):
            wrong_dict[c].append(tid)
    k+=1

0
100
200
300
400
500
600
700
800
900


In [27]:
wrong_stateavg=df_union_before_and_after[df_union_before_and_after['_tid_'].isin(wrong_dict['Stateavg'])]

In [28]:
wrong_stateavg

,Address1,Address2,Address3,City,Condition,CountyName,EmergencyService,HospitalName,HospitalOwner,HospitalType,...,MeasureName,PhoneNumber,ProviderNumber,Sample,Score,State,Stateavg,ZipCode,_tid_,type
864,2801 debarr road,NaN,NaN,anchorage,heart failure,anchorage,yes,alaska regional hospital,proprietary,acute care hospitals,...,heart failure patients given discharge instruc...,9072761131,20017,57 patients,91%,ak,al_hf-1,99508,432.0,after_clean
865,2801 debarr road,NaN,NaN,anchorage,heart failure,anchorage,yes,alaska regional hospital,proprietary,acute care hospitals,...,heart failure patients given discharge instruc...,9072761131,20017,57 patients,91%,ak,ak_hf-1,99508,432.0,before_clean
866,2801 debarr road,NaN,NaN,anchorage,heart failure,anchorage,yes,alaska regional hospital,proprietary,acute care hospitals,...,heart failure patients given an evaluation of ...,9072761131,20017,61 patients,98%,ak,al_hf-2,99508,433.0,after_clean
867,2801 debarr road,NaN,NaN,anchorage,heart failure,anchorage,yes,alaska regional hospital,proprietary,acute care hospitals,...,heart failure patients given an evaluation of ...,9072761131,20017,61 patients,98%,ak,ak_hf-2,99508,433.0,before_clean
870,2801 debarr road,NaN,NaN,anchorage,heart failure,anchorage,yes,alaska regional hospital,proprietary,acute care hospitals,...,heart failure patients given smoking cessation...,9072761131,20017,13 patients,100%,ak,ak_hf-4,99508,435.0,before_clean
871,2801 debarr road,NaN,NaN,anchorage,heart failure,anchorage,yes,alaska regional hospital,proprietary,acute care hospitals,...,heart failure patients given smoking cessation...,9072761131,20017,13 patients,100%,ak,al_hf-4,99508,435.0,after_clean
872,2801 debarr road,NaN,NaN,anchorage,pneumonia,anchorage,yes,alaska regional hospital,proprietary,acute care hospitals,...,pneumonia patients assessed and given pneumoco...,9072761131,20017,37 patients,97%,ak,ak_pn-2,99508,436.0,before_clean
873,2801 debarr road,NaN,NaN,anchorage,pneumonia,anchorage,yes,alaska regional hospital,proprietary,acute care hospitals,...,pneumonia patients assessed and given pneumoco...,9072761131,20017,37 patients,97%,ak,al_pn-2,99508,436.0,after_clean
874,2801 debarr road,NaN,NaN,anchoraxe,pneumonia,anchorage,yes,alaska regional hospital,proprietary,acute care hospitals,...,pneumonia patients whose initial emergency roo...,9072761131,20017,47 patients,100%,ak,ak_pn-3b,99508,437.0,before_clean
875,2801 debarr road,NaN,NaN,anchoraxe,pneumonia,anchorage,yes,alaska regional hospital,proprietary,acute care hospitals,...,pneumonia patients whose initial emergency roo...,9072761131,20017,47 patients,100%,ak,al_pn-3b,99508,437.0,after_clean


In [29]:
wrong_stateavg.sort_values(by=['_tid_','type','Stateavg'], inplace=True)

/home/opc/chenjie/label/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [30]:
wrong_stateavg[['Stateavg', '_tid_', 'type']]

,Stateavg,_tid_,type
864,al_hf-1,432.0,after_clean
865,ak_hf-1,432.0,before_clean
2432,ak_hf-1,432.0,ground_truth
866,al_hf-2,433.0,after_clean
867,ak_hf-2,433.0,before_clean
2433,ak_hf-2,433.0,ground_truth
871,al_hf-4,435.0,after_clean
870,ak_hf-4,435.0,before_clean
2435,ak_hf-4,435.0,ground_truth
873,al_pn-2,436.0,after_clean


In [31]:
conn.close()

### given an attribute and a tid (row id), find responsible rule(s)

In [32]:
import hc_responsibility
hc_responsibility.rule_responsibility(attr_name='Stateavg', tid=432)

TypeError: rule_responsibility() missing 1 required positional argument: 'size'